In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split

# CSV-Datei laden
df = pd.read_csv('PremierLeague.csv', sep=';')

# Tore bisher berechnen
from collections import defaultdict
df = df.sort_values(['Season', 'MatchWeek', 'Date']).reset_index(drop=True)
goals_scored = defaultdict(lambda: defaultdict(int))
home_goals_sofar = []
away_goals_sofar = []

for _, row in df.iterrows():
    season = row['Season']
    home = row['HomeTeam']
    away = row['AwayTeam']
    home_goals_sofar.append(goals_scored[season][home])
    away_goals_sofar.append(goals_scored[season][away])
    goals_scored[season][home] += row['FullTimeHomeTeamGoals']
    goals_scored[season][away] += row['FullTimeAwayTeamGoals']

df['HomeGoalsSoFar'] = home_goals_sofar
df['AwayGoalsSoFar'] = away_goals_sofar
df['Heimvorteil'] = 1  # Heimteam hat immer Heimvorteil

# Zielvariable umwandeln
df['Result'] = df['FullTimeResult'].map({'H': 'Heimsieg', 'D': 'Unentschieden', 'A': 'Auswärtssieg'})

# Kategorische Werte codieren
from sklearn.preprocessing import LabelEncoder

le_team = LabelEncoder()
teams = pd.concat([df['HomeTeam'], df['AwayTeam']])
le_team.fit(teams)
df['Home_enc'] = le_team.transform(df['HomeTeam'])
df['Away_enc'] = le_team.transform(df['AwayTeam'])

le_result = LabelEncoder()
df['Result_enc'] = le_result.fit_transform(df['Result'])

# Features und Ziel definieren
X = df[['Home_enc', 'Away_enc', 'HomeGoalsSoFar', 'AwayGoalsSoFar', 'Heimvorteil']]
y = df['Result_enc']

# Trainings- und Testdaten Split (90/10)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, random_state=42)


In [31]:
from sklearn.ensemble import RandomForestClassifier

# Modell erstellen und trainieren
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

Ich habe mich für den RandomForestClassifier entschieden, da diese Methode viele Entscheidungsbäume kombiniert und die Vorhersagen dann, die Entscheidung mit einem Mehrheitsentscheid dieser Entscheidungsbäume abgibt. Dies ist nützlich bei einer Vorhersage wie meiner mit den Fussballspielen, weil es im Fussball viele verschiedene Einflüsse gibt, die dem Spiel wie Endstand der letzten Partien dieser beiden Mannschaften, Heim-Auswärtsvorteil, wie viele Tore eine Mannschaft bisher geschossen hat in der Saison.

In [33]:
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt

# Vorhersagen auf Testdaten
y_pred = model.predict(X_test)

# Accuracy berechnen
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy auf Testdaten: {accuracy:.3f}")



# Einige Beispielvorhersagen ausgeben
predictions = pd.DataFrame({
    'Heimteam': le_team.inverse_transform(X_test['Home_enc']),
    'Auswärtsteam': le_team.inverse_transform(X_test['Away_enc']),
    'Tatsächlich': le_result.inverse_transform(y_test),
    'Vorhersage': le_result.inverse_transform(y_pred)
})

print(predictions.head(10))


Accuracy auf Testdaten: 0.453
      Heimteam      Auswärtsteam    Tatsächlich    Vorhersage
0       Fulham  Sheffield United       Heimsieg      Heimsieg
1      Chelsea         Newcastle       Heimsieg      Heimsieg
2     West Ham           Chelsea  Unentschieden  Auswärtssieg
3  Southampton         Leicester   Auswärtssieg      Heimsieg
4     Man City     Nott'm Forest       Heimsieg      Heimsieg
5        Derby           Chelsea   Auswärtssieg  Auswärtssieg
6      Everton           Watford       Heimsieg      Heimsieg
7        Stoke         Liverpool   Auswärtssieg  Auswärtssieg
8    Liverpool        Birmingham       Heimsieg      Heimsieg
9    West Brom          Man City   Auswärtssieg      Heimsieg


Mit einer Genauigkeit von 45,3 % ist mein Modell besser, als ich erwartet hätte. Man kann erkennen, dass eine Vorhersage, dass das Heimteam gewinnt, am meisten stimmt, jedoch ist das Unentschieden schwierig vorherzusagen. Man könnte den Score noch verbessern, indem man die aktuelle Tabellenplatzierung oder etwa die letzten 5 Spiele einbezieht.
